In [204]:
import requests
import re
import pandas as pd
import io
import html5lib
from bs4 import BeautifulSoup as bs
from IPython.display import clear_output
import platform

Any urls we may need

In [205]:
_BASE_URL = 'https://verd.lyfjastofnun.is/'
_URL = 'https://verd.lyfjastofnun.is/?pageid=10'

Fetch the urls of all excel files, do not get xlsx files that were uploaded due to them being a security issue.

https://stackoverflow.com/questions/65250207/pandas-cannot-open-an-excel-xlsx-file

In [206]:
r = requests.get(_URL)
soup = bs(r.text)
#xlsx
_links = soup.find_all('a', {'href': re.compile(r'verdskra.*\.xls$')})

Iterate through all excel files and add to our dataframe for processing.

Skip the first file as it simply explains the columns of the rest of the files.

In [210]:
appended_data = []
for index, link in enumerate(_links[1:]):
    price_link = link.get("href")
    file_name = re.search('gogn/(.*).xls',price_link).group(1)
    response = requests.get(_BASE_URL + price_link).content#
    inc_data = pd.read_excel(response).assign(file=file_name)
    appended_data.append(inc_data)

data = pd.concat(appended_data)
data.head(10)

,Norrænt vörunúmer,Lyfjaheitisnúmer,Heiti lyfs,Form lyfs,Styrkur,Styrkeining,Magn,Magneining,Fj. Pakkninga,Pakkning,...,"Skammtur að hámarki fyrir fjölda daga (100, 30)",Heildsöluverð,Smásöluverð,Meðferð,Hmagn,"Á undanþágulista (1, 0)",DDD,Unnamed: 1,Afskáningardags,Unnamed: 36
0,379875.0,N05BA0105,Stesolid,eþ-lausn,5,"mg/2,5ml","2,5",ml,5.0,túpa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,379883.0,N05BA0105,Stesolid,eþ-lausn,10,"mg/2,5ml","2,5",ml,5.0,túpa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,43729.0,N05CD0503,Halcion,töflur,"0,125",mg,30,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,48199.0,A03CA0201,Librax,töflur,NaN,NaN,100,stk,1.0,glas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,52286.0,N05BA1205,Alprazolam Mylan,töflur,"0,25",mg,20,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
5,52297.0,N05BA1205,Alprazolam Mylan,töflur,"0,25",mg,50,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
6,52309.0,N05BA1205,Alprazolam Mylan,töflur,"0,5",mg,20,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
7,52320.0,N05BA1205,Alprazolam Mylan,töflur,"0,5",mg,50,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
8,52331.0,N05BA1205,Alprazolam Mylan,töflur,"0,25",mg,100,stk,1.0,glas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
9,70203.0,N05CF0107,Imomed,filmhtfl,"7,5",mg,30,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


Time to clean the data as best we can. Lets merge all the data.

In [216]:
data_clean = data
#data = data_clean

In [212]:
data['nvnr'] = data['Norrænt vörunúmer'].combine_first(data['Nvnr'])
data['form'] = data['Form lyfs'].combine_first(data['Form'])
data['name'] = data['Heiti lyfs'].combine_first(data['Heiti'])
data['package_quantity'] = data['Fj. Pakkninga'].combine_first(data['Fjöldi pakkninga'])
data['dentist_prescription'] = data['Tannlæknir ávísar'].combine_first(data['Tannlæknir ávísar (1, 0)'])
data['vet_prescription'] = data['Dýralæknir ávísar'].combine_first(data['Dýralæknir ávísar  (1, 0)'])
data['specialist_prescription'] = data['Sérfræðingur ávísar'].combine_first(data['Sérfræðingur ávísar  (1, 0)'])
data['market_authorization_holder_code'] = data['Markaðsleyfishafi kóði'].combine_first(data['Markaðsleyfishafi-Kódi'])
data['hospital'] = data['Sjúkrahúslyf'].combine_first(data['Sjúkrahúslyf  (1, 0)'])
data['exception_list'] = data['Á undanþágulista'].combine_first(data['Á undanþágulista (1, 0)'])
data['package_quantity'] = data['package_quantity'].combine_first(data['Fj í pakkningu'])
data['copay_availability'] = data['Greiðsluþátttökuverð'].combine_first(data['Greiðsluþátttökusmásöluverð'])
data['permitted_dosages'] = data['Skammtur, hámark daga'].combine_first(data['Skammtur að hámarki fyrir fjölda daga  (100,  30)'])
data['daily_dosage'] = data['DDD á pakkningu'].combine_first(data['DDD'])
data['strength'] = data['Styrkur'].combine_first(data['Tölugildi styrks'])
data['strength_unit'] = data['Styrkeining'].combine_first(data['Einingar styrks'])
data['strength_unit'] = data['strength_unit'].combine_first(data['Einstyrks'])
data['strength_unit'] = data['strength_unit'].combine_first(data['Eining styrks'])
data['amount'] = data['Magn'].combine_first(data['Tölugildi magns'])
data['amount_unit'] = data['Magneining'].combine_first(data['Einingar magns'])
data['amount_unit'] = data['amount_unit'].combine_first(data['Einmagns'])
data['total_amount'] = data['Heildarmagn'].combine_first(data['Hmagn'])
data['therapy_instructions'] = data['Meðferð'].combine_first(data['Leiðbeiningar um meðferð'])
data['shelf_life'] = data['Geymsluþol'].combine_first(data['Geymsluþol  (F,H,K,S,A)'])
data['comparable_currency'] = data['Viðmiðunargjaldmiðill'].combine_first(data['Viðmiðunargjald'])
data['included_in_insurance'] = data['Ákvörðun LGN um greiðsluþátttöku'].combine_first(data['Ákvörðun LGN'])
data['copay_group'] = data['Greiðsluhlutur SÍ'].combine_first(data['Greiðsluhlutur TR'])
data['antipsychotic'] = data['Geðlyf á geðdeild'].combine_first(data['Geðlyf á geðdeild  (1, 0)'])
data['regulated'] = data['Eftirritunarskylt'].combine_first(data['Eftirritunarskylt  (1, 0)'])
data['registrated_at_date'] = data['Skráningardagsetning'].combine_first(data['Skráningardags'])
data['unregistrated_at_date'] = data['Afskráningardagsetning'].combine_first(data['Afskáningardags'])
data['unregistrated_at_date'] = data['unregistrated_at_date'].combine_first(data['Afskráning'])

## Drop our merged columns
data = data.drop(columns=['Norrænt vörunúmer','Nvnr',
'Form lyfs','Form',
'Fj. Pakkninga','Fjöldi pakkninga','Fj í pakkningu',
'Tannlæknir ávísar','Tannlæknir ávísar (1, 0)',
'Heiti lyfs','Heiti',
'Dýralæknir ávísar','Dýralæknir ávísar  (1, 0)',
'Markaðsleyfishafi kóði','Markaðsleyfishafi-Kódi',
'Sérfræðingur ávísar  (1, 0)','Sérfræðingur ávísar',
'Sjúkrahúslyf','Sjúkrahúslyf  (1, 0)',
'Á undanþágulista','Á undanþágulista (1, 0)',
'Greiðsluþátttökuverð','Greiðsluþátttökusmásöluverð',
'Skammtur, hámark daga','Skammtur að hámarki fyrir fjölda daga  (100,  30)',
'DDD','DDD á pakkningu',
'Styrkur','Styrkeining','Einstyrks','Einingar styrks','Eining styrks','Tölugildi styrks',
'Magn','Magneining','Einmagns','Einingar magns','Tölugildi magns',
'Heildarmagn','Hmagn',
'Meðferð','Leiðbeiningar um meðferð',
'Unnamed: 0','Unnamed: 1','Unnamed: 36',
'Viðmiðunargjaldmiðill','Viðmiðunargjald',
'Ákvörðun LGN um greiðsluþátttöku','Ákvörðun LGN',
'Greiðsluhlutur SÍ','Greiðsluhlutur TR',
'Geðlyf á geðdeild','Geðlyf á geðdeild  (1, 0)',
'Eftirritunarskylt','Eftirritunarskylt  (1, 0)',
'Skráningardagsetning','Skráningardags',
'Afskráning','Afskráningardagsetning','Afskáningardags'])


KeyError: 'Norrænt vörunúmer'

In [213]:
## Rename our leftover columns
data = data.rename(columns={
'Lyfjaheitisnúmer':'drug_number',
'Pakkning':'packaging',
'Athugasemdir':'comments',
'ATC-flokkur':'ATC',
'Afgreiðslumáti':'delivery_method_code',
'Skömmtunarmöguleikar ':'prescription_available',
'Öryggiskröfur LST':'security_standards_LST',
'Greiðsluhlutur SÍ':'copay_amount',
'Leyfisskyld lyf ':'prescription_required',
'Hámarks heildsöluverð':'maximum_wholesale_price',
'Hámarks smásöluverð':'maximum_retail_price',
'Álagningarflokkur':'pay_group',
'Viðmiðunarverðflokkur':'comparable_price_range',
'Viðmiðunarverð':'comparable_price',
'Staða lyfs ':'status',
'Umboðsaðili':'commissioner',
'Markaðsleyfishafi':'market_authorization_holder',
'Heiti virka innihaldsefnis':'active_ingredient',
'Lyfjaframleiðenda heildsöluverð':'manufacturer_wholesale_price',
'Umboðsmanna heildsöluverð':'commissioner_wholesale_price',
'Heildsölu heildsöluverð':'wholesaler_wholesale_price',
'Innkaupsverð apóteka':'pharmacy_buy_price',
'Fellt úr lyfjaverðskrá vegna birgðaskorts':'dropped_from_sale_due_to_shortage',
'Heildsali':'wholesaler',
'H merking':'healthcare_specific',
'Ávana og fíknilyf':'addictive_medicine',
'Ávana og fíknilyf magn':'addictive_medicine_amount',
'Ávana og fíknilyf eining':'addictive_medicine_unit',
'Hjúkr-/ljósm ávísar':'wet_nurse_prescription',
'GTIN númer':'GTIN',
'Geymsluskilyrði':'storage_requirement'
})

In [214]:
print('number of columns are now ', len(data.columns), 'They were ', len(data_clean.columns))
data.columns

number of columns are now  71 They were  71


Index(['drug_number', 'packaging', 'comments', 'ATC', 'registrated_at_date',
       'unregistrated_at_date', 'delivery_method_code',
       'prescription_available', 'security_standards_LST',
       'prescription_required', 'maximum_wholesale_price',
       'maximum_retail_price', 'pay_group', 'comparable_price_range',
       'comparable_price', 'status', 'commissioner',
       'market_authorization_holder', 'active_ingredient',
       'manufacturer_wholesale_price', 'commissioner_wholesale_price',
       'wholesaler_wholesale_price', 'pharmacy_buy_price',
       'dropped_from_sale_due_to_shortage', 'wholesaler',
       'healthcare_specific', 'addictive_medicine',
       'addictive_medicine_amount', 'addictive_medicine_unit',
       'wet_nurse_prescription', 'file', 'regulated', 'GTIN',
       'storage_requirement', 'Geymsluþol', 'antipsychotic',
       'Markaðsfyrirtækja/umboðsmanna heildsöluverð', 'Mismunur',
       'Skráningardags', 'Afskráning', 'Afgreiðslumáti  (R,L,U)',
       'G

In [215]:
data.to_csv("output.txt")